In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
train=pd.read_csv('train_users_2.csv')

In [7]:
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [8]:
test=pd.read_csv('test_users.csv')

In [9]:
country=pd.read_csv('countries.csv')

In [10]:
age=pd.read_csv('age_gender_bkts.csv')

In [11]:
sessions=pd.read_csv('sessions.csv')

In [12]:
age.shape

(420, 5)

In [13]:
country.shape

(10, 7)

In [14]:
sessions.shape

(10567737, 6)

In [15]:
test.shape

(62096, 15)

In [16]:
train.shape

(213451, 16)

In [17]:
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [18]:
train.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination'],
      dtype='object')

In [19]:
test.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser'],
      dtype='object')

In [20]:
data=pd.concat([train[['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser',]],test])

In [21]:
data.shape

(275547, 15)

In [22]:
data.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         186639
gender                          0
age                        116866
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser                   0
dtype: int64

In [23]:
data.drop(columns='date_first_booking',inplace=True)

In [24]:
data.drop(columns='age',inplace=True)

In [25]:
data.first_affiliate_tracked.value_counts()

first_affiliate_tracked
untracked        143181
linked            62064
omg               54859
tracked-other      6655
product            2353
marketing           281
local ops            69
Name: count, dtype: int64

In [26]:
data.fillna(value='untracked',inplace=True)

In [27]:
data.isnull().sum()

id                         0
date_account_created       0
timestamp_first_active     0
gender                     0
signup_method              0
signup_flow                0
language                   0
affiliate_channel          0
affiliate_provider         0
first_affiliate_tracked    0
signup_app                 0
first_device_type          0
first_browser              0
dtype: int64

In [28]:
data['signup_flow']=data.signup_flow.apply(lambda x:np.log(x+1))

In [29]:
data.affiliate_provider.value_counts()

affiliate_provider
direct                 181270
google                  65956
other                   13036
facebook                 3996
bing                     3719
craigslist               3475
padmapper                 836
vast                      830
yahoo                     653
facebook-open-graph       566
gsp                       455
meetup                    358
email-marketing           270
naver                      66
baidu                      32
yandex                     18
wayn                        8
daum                        3
Name: count, dtype: int64

In [30]:
data.signup_flow

0        0.000000
1        0.000000
2        1.386294
3        0.000000
4        0.000000
           ...   
62091    0.000000
62092    3.178054
62093    0.000000
62094    0.000000
62095    0.000000
Name: signup_flow, Length: 275547, dtype: float64

In [31]:
data.columns

Index(['id', 'date_account_created', 'timestamp_first_active', 'gender',
       'signup_method', 'signup_flow', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser'],
      dtype='object')

data['year_timestamp_first_active']=data.timestamp_first_active.astype('str').apply(lambda x:(datetime.datetime.strptime(x,'%Y%m%d%H%M%S').year))
data['month_timestamp_first_active']=data.timestamp_first_active.astype('str').apply(lambda x:(datetime.datetime.strptime(x,'%Y%m%d%H%M%S').month))
data['day_timestamp_first_active']=data.timestamp_first_active.astype('str').apply(lambda x:(datetime.datetime.strptime(x,'%Y%m%d%H%M%S').day))
data['hour_timestamp_first_active']=data.timestamp_first_active.astype('str').apply(lambda x:(datetime.datetime.strptime(x,'%Y%m%d%H%M%S').hour))
data['minute_timestamp_first_active']=data.timestamp_first_active.astype('str').apply(lambda x:(datetime.datetime.strptime(x,'%Y%m%d%H%M%S').minute))
data['second_timestamp_first_active']=data.timestamp_first_active.astype('str').apply(lambda x:(datetime.datetime.strptime(x,'%Y%m%d%H%M%S').second))
data.drop(columns='timestamp_first_active',inplace=True)

data['year_date_account_created']=data.date_account_created.apply(lambda x:int(x.split('-')[0]))
data['month_date_account_created']=data.date_account_created.apply(lambda x:int(x.split('-')[1]))
data['day_date_account_created']=data.date_account_created.apply(lambda x:int(x.split('-')[2]))
data.drop(columns='date_account_created',inplace=True)

data.to_csv(r'parsed_csv.csv')


In [32]:
data=pd.read_csv('parsed_csv.csv')
data.drop(columns='Unnamed: 0',inplace=True)

In [33]:
data.head()

,id,gender,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,year_timestamp_first_active,month_timestamp_first_active,day_timestamp_first_active,hour_timestamp_first_active,minute_timestamp_first_active,second_timestamp_first_active,year_date_account_created,month_date_account_created,day_date_account_created
0,gxn3p5htnn,-unknown-,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2009,3,19,4,32,55,2010,6,28
1,820tgsjxq7,MALE,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2009,5,23,17,48,9,2011,5,25
2,4ft3gnwmtx,FEMALE,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2009,6,9,23,12,47,2010,9,28
3,bjjt8pjhuk,FEMALE,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2009,10,31,6,1,29,2011,12,5
4,87mebub9p4,-unknown-,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2009,12,8,6,11,5,2010,9,14


In [34]:
data.isnull().sum()

id                               0
gender                           0
signup_method                    0
signup_flow                      0
language                         0
affiliate_channel                0
affiliate_provider               0
first_affiliate_tracked          0
signup_app                       0
first_device_type                0
first_browser                    0
year_timestamp_first_active      0
month_timestamp_first_active     0
day_timestamp_first_active       0
hour_timestamp_first_active      0
minute_timestamp_first_active    0
second_timestamp_first_active    0
year_date_account_created        0
month_date_account_created       0
day_date_account_created         0
dtype: int64

In [35]:
data=pd.read_csv('dummed_features.csv')

In [36]:
data.drop(columns='id',inplace=True)

## Categorical to numerical

In [37]:
# Gender

data=pd.get_dummies(data=data,columns=['gender','first_device_type', 'first_browser','affiliate_channel','affiliate_provider','signup_app','signup_method','language','first_affiliate_tracked'],prefix='Dum')

data.columns

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275547 entries, 0 to 275546
Columns: 146 entries, Unnamed: 0 to Dum_untracked
dtypes: int64(146)
memory usage: 306.9 MB


data.to_csv(r'dummed_features.csv')

In [39]:
train.country_destination.value_counts()

country_destination
NDF      124543
US        62376
other     10094
FR         5023
IT         2835
GB         2324
ES         2249
CA         1428
DE         1061
NL          762
AU          539
PT          217
Name: count, dtype: int64

In [40]:
cate=train.country_destination.astype('category').cat.categories.tolist()

In [41]:
cate

['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NDF', 'NL', 'PT', 'US', 'other']

In [42]:
replace_map={'country_destination':dict(zip(cate,list(range(1,len(cate)+1))))}

In [43]:
replace_map

{'country_destination': {'AU': 1,
  'CA': 2,
  'DE': 3,
  'ES': 4,
  'FR': 5,
  'GB': 6,
  'IT': 7,
  'NDF': 8,
  'NL': 9,
  'PT': 10,
  'US': 11,
  'other': 12}}

In [44]:
train.replace(replace_map,inplace=True)

/tmp/ipykernel_7353/2074875287.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.replace(replace_map,inplace=True)


In [45]:
train.country_destination

0          8
1          8
2         11
3         12
4         11
          ..
213446     8
213447     8
213448     8
213449     8
213450     8
Name: country_destination, Length: 213451, dtype: int64

In [46]:
test.shape

(62096, 15)

In [47]:
train.shape

(213451, 16)

In [48]:
x_train=data[:213451]

In [49]:
x_train.shape

(213451, 146)

In [50]:
y_train=train.country_destination

In [51]:
y_train.shape

(213451,)

In [52]:
x_test=data[213451:]

In [53]:
test.shape

(62096, 15)

In [54]:
x_test.shape

(62096, 146)

In [55]:
x_train.tail(1)


,Unnamed: 0,signup_flow,year_timestamp_first_active,month_timestamp_first_active,day_timestamp_first_active,hour_timestamp_first_active,minute_timestamp_first_active,second_timestamp_first_active,year_date_account_created,month_date_account_created,...,Dum_th,Dum_tr,Dum_zh,Dum_linked,Dum_local ops,Dum_marketing,Dum_omg,Dum_product,Dum_tracked-other,Dum_untracked
213450,213450,25,2014,6,30,23,58,24,2014,6,...,0,0,0,0,0,0,0,0,0,1


In [56]:
train.tail(1)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
213450,nw9fwlyb5f,2014-06-30,20140630235824,NaN,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,8


In [57]:
x_test.head(1)

,Unnamed: 0,signup_flow,year_timestamp_first_active,month_timestamp_first_active,day_timestamp_first_active,hour_timestamp_first_active,minute_timestamp_first_active,second_timestamp_first_active,year_date_account_created,month_date_account_created,...,Dum_th,Dum_tr,Dum_zh,Dum_linked,Dum_local ops,Dum_marketing,Dum_omg,Dum_product,Dum_tracked-other,Dum_untracked
213451,213451,0,2014,7,1,0,0,6,2014,7,...,0,0,0,0,0,0,0,0,0,1


In [58]:
test.head(1)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari


## Machine Learning

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

### Random Forest

In [60]:
rfs=RandomForestClassifier()

In [61]:
rff=rfs.fit(x_train,y_train)

In [62]:
y_predict=rff.predict(x_test)

In [63]:
y_predic=pd.DataFrame(y_predict)

In [64]:
reverse_map={0:dict(zip(list(range(1,len(cate)+1)),cate))}
y_predic.replace(reverse_map,inplace=True)

In [65]:
# Saving the results in Csv
data={'id':test.id,'country':y_predic[0]}
pd.DataFrame(data).set_index('id').to_csv(r'submission files/submission_RF01.csv')

OSError: Cannot save file into a non-existent directory: 'submission files'

In [ ]:
best_score=pd.read_csv(r'submission files/submission_RF01.csv')

### Support Vector Machine

In [ ]:
sv=svm()

In [ ]:
svr=rfs.fit(x_train_f,y_train)

In [ ]:
y_predict=svr.predict(x_test_f)

In [ ]:
y_predict

In [ ]:
# Saving the results in Csv
data={'Id':range(1461,2920),'SalePrice':y_predict}
pd.DataFrame(data).set_index('Id').to_csv(r'submission files/submission_RF02.csv')

In [ ]:
best_score=pd.read_csv(r'submission files/submission_RF01.csv')

In [ ]:
a=best_score['SalePrice']-y_predict

In [ ]:
#fig,ax1=plt.subplot(1,1)
sns.distplot(a,bins=30)


In [ ]:
best_score['new']=y_predict

In [ ]:
sns.heatmap(best_score[['SalePrice','new']].corr(),annot=True)

### K-Nearest Neighbours

In [ ]:
KNN=KNeighborsClassifier()

In [ ]:
knn=KNN.fit(x_train,y_train)

In [ ]:
y_predict=knn.predict(x_test)

In [ ]:
y_predic=pd.DataFrame(y_predict)

In [ ]:
reverse_map={0:dict(zip(list(range(1,len(cate)+1)),cate))}
y_predic.replace(reverse_map,inplace=True)

In [ ]:
# Saving the results in Csv
data={'id':test.id,'country':y_predic[0]}
pd.DataFrame(data).set_index('id').to_csv(r'submission files/submission_KNN01.csv')

In [ ]:
y_predict

In [ ]:
# Saving the results in Csv
data={'Id':range(1461,2920),'SalePrice':y_predict}
pd.DataFrame(data).set_index('Id').to_csv(r'submission files/submission_RF02.csv')

In [ ]:
best_score=pd.read_csv(r'submission files/submission_RF01.csv')

In [ ]:
a=best_score['SalePrice']-y_predict

In [ ]:
#fig,ax1=plt.subplot(1,1)
sns.distplot(a,bins=30)


In [ ]:
best_score['new']=y_predict

In [ ]:
sns.heatmap(best_score[['SalePrice','new']].corr(),annot=True)

### XGBoostClassifier

In [ ]:
xg=XGBClassifier()

In [ ]:
xgf=xg.fit(x_train,y_train)

In [ ]:
y_predict=xgf.predict(x_test)

In [ ]:
y_predic=pd.DataFrame(y_predict)

In [ ]:
reverse_map={0:dict(zip(list(range(1,len(cate)+1)),cate))}
y_predic.replace(reverse_map,inplace=True)

In [ ]:
# Saving the results in Csv
data={'id':test.id,'country':y_predic[0]}
pd.DataFrame(data).set_index('id').to_csv(r'submission files/submission_XG01.csv')